# Jacobian Pseudo Inverse Controller
ToDo: Add theory of operation

## Run In Simulation

In [ ]:
import time
import numpy as np
from FR3Py.controllers.waypoint_controller import WaypointController
from FR3Py.envs.pybullet import FR3Sim

robot = FR3Sim(render_mode='human', mode='velocity', Ts=1/1000)
robot.reset()
controller = WaypointController(kp=2)
while True:
    q, dq = robot.get_state()
    dq_cmd = controller.compute(q, dq)
    robot.step(dq_cmd)
    time.sleep(1/1000)

## Run With Real Robot

In [ ]:
from FR3Py.interfaces import FR3Real
robot = FR3Real()

In [ ]:
import time
import numpy as np
from FR3Py.controllers.waypoint_controller import WaypointController
controller = WaypointController(kp=1.5)
# Read the initila pose of the robot
time.sleep(1)
state = robot.get_state()

q, dq = state['q'], state['dq']
p0 = controller.robot.getInfo(q,dq)['P_EE']
R0 = controller.robot.getInfo(q,dq)['R_EE']
T0 = np.vstack([np.hstack([R0, p0.reshape(3,1)]), np.array([0,0,0,1])])
start_time = time.time()
while True:
    time.sleep(0.01)
    t = time.time()-start_time
    x = 0.1*np.sin(t)
    T= T0@np.vstack([np.hstack([np.eye(3), np.array([0,0,x]).reshape(3,1)]), np.array([0,0,0,1])])
    state = robot.get_state()
    q, dq = state['q'], state['dq']
    cmd = controller.compute(q,dq, T_cmd=T)
    robot.send_joint_command(cmd[0:7])

# TF Pose Capture Node

Run the following cell to repeatedly read the pose of the wall published as a TF message and set it to a global valirable pose:

In [ ]:
import rospy
import tf
import threading

def tf_listener_thread():
    global pose

    rospy.init_node("tf_listener_node")
    listener = tf.TransformListener()

    # the frame where the data originated
    source_frame = "/camera_color_optical_frame"

    # the frame where the data should be transformed
    target_frame = "/wall"

    listener.waitForTransform(target_frame, source_frame, rospy.Time(), rospy.Duration(4.0))

    # Main loop to continuously update the pose
    while not rospy.is_shutdown():
        try:
            (trans, rot) = listener.lookupTransform(target_frame, source_frame, rospy.Time(0))
            pose = (trans, rot)
            rospy.sleep(0.1)  # Adjust the sleep duration as needed
        except (tf.LookupException, tf.ConnectivityException, tf.ExtrapolationException):
            rospy.logwarn("Error occurred while retrieving TF transform.")

# Initialize the global pose variable
pose = None

# Create and start the thread
tf_thread = threading.Thread(target=tf_listener_thread)
tf_thread.start()